In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Import necessary libraries
import pandas as pd 
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

titanic2 = pd.read_csv("../input/titanic/train_and_test2.csv").dropna()

data = titanic2[['Age', 'Fare', 'Sex', 'sibsp','Parch','Pclass','Embarked','2urvived']]
X = titanic2[['Age', 'Fare', 'Sex', 'sibsp','Parch','Pclass','Embarked']]
y = titanic2['2urvived']
#There were two incomplete rows in the original dataset. dropna() should remove them.
X[X.isnull().any(axis=1)]



Il dataset in input ha ricevuto alcune operazioni di preprocessing rispetto a quello del contest originale:

1. I set di test e train sono stati riuniti
2. Gli elementi testuali come il sesso sono gia stati convertiti in indici numerici in base al loro label.
3. Alcune categorie poco attinenti (come il nome della persona) sono state escluse.

Mostriamo innanzitutto la correlazione tra le colonne di X e il risultato, e l'autocorrelazione tra gli elementi di X.
L'operazione viene svolta tramite il preset "heatmap" di seaborn

In [ ]:
# Verify correlation with 'Survived' and within X
plt.figure(figsize=(12,10))
cor = data.corr()
sbn.heatmap(cor, annot=True, cmap=plt.cm.Spectral)
plt.show()

Non è sorprendente vedere uno sbilanciamento in correlazione tra il sesso del passeggero e la sua capacita di sopravvivenza.
Circa il 70% delle donne a bordo sopravvissero il naufragio, contro il 20% degli uomini.

Dal grafico si possono estrarre anche informazioni di autocorrelazione da cui migliorare l'input al modello naive Bayes, che si aspetta 
variabili relativamente indipendenti. La più evidente è Fare/Pclass: il costo del biglietto e la classe a bordo sono ovviamente fortemente correlati.
Testando quale delle due classi genera un risultato migliore, si puo vedere che Pclass fornisce informazioni più utili alla classificazione corretta se unita alle altre colonne rimanenti.

In [ ]:
# Unsurprisingly, boarding class and fare are strongly correlated. 
# Less correlated but still quite high are Age/Class and Siblings+Spouses/Parents+children
# Fare might give us more granular information between the two.
# On the other hand, information about the actual class has stronger implication on the person's location within the ship.
# The model seems to perform much better with 'Fare' dropped.

#X =X.drop(['Pclass'], axis=1)
X = X.drop('Fare', axis=1)


In [ ]:
# Split into training and test with a fixed random state.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix

def evaluate(y_test, y_pred):
        print("accuracy:", accuracy_score(y_test, y_pred))
        print("precision:", precision_score(y_test, y_pred))
        print("recall:", recall_score(y_test, y_pred))
        #print("AUC:", roc_auc_score(y_test, y_pred))

Applichiamo sullo stesso split due modelli separati:
* Un modello generico di GaussianNB, che scala e tratta i dati assumendo che siano generalmente distribuiti secondo una distribuzione normale
* Un modello ComplementNB, che sklearn dichiara empiricamente più adatto a trattare dati sbilanciati (come nel caso del titanic, contro i sopravvissuti)

Stampiamo anche le matrici di confusione e le metriche corrispondenti delle predizioni per dimostrare gli effetti dei modelli individuali.

In [ ]:
# Apply scikit-learn's GaussianNB model.
from sklearn.naive_bayes import GaussianNB

# No need to preprocess data since GaussianNB will by default rescale and fit the input on its own.
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

evaluate(y_test, y_pred)

# Plot the confusion matrix for the test sample [0 for died, 1 for survived].

errors = confusion_matrix(y_test,y_pred)
print(errors)
sbn.heatmap(errors, annot=True, fmt="d", cmap=plt.cm.Reds)

y_gauss = y_pred

In [ ]:
# The total accuracy is decent (especially considering the original titanic contest winners' was also quite low)
# But recall and precision could be better. It may be because the dataset is imbalanced against survivors (at about 70%)
# Let's try out Complement Naive Bayes, since sklearn's docs say 
# 'CNB is an adaptation of the standard multinomial naive Bayes (MNB) algorithm that is particularly suited for imbalanced data sets. '

from sklearn.naive_bayes import ComplementNB

cnb = ComplementNB() # No particular default parameter tweaking needed.
cnb.fit(X_train, y_train)
y_pred = cnb.predict(X_test)

evaluate(y_test, y_pred)

errors = confusion_matrix(y_test,y_pred)
print(errors)
sbn.heatmap(errors, annot=True, fmt="d", cmap=plt.cm.Reds)

y_compl = y_pred

Possiamo notare che nonostante l'accuratezza del modello GaussianNB sia relativamente alta, genera più falsi negativi che positivi veri.
Questo viene evidenziato dal punteggio di recall molto basso.

Il risultato del modello ComplementNB perde leggermente nell'accuratezza totale ma discrimina in maniera molto più accurata tra i sopravvissuti.
Ciò può indicare che l'errore di bias verso una direzione nelle predizioni del modello gaussiano fosse effettivamente generato dallo squilibrio del dataset.

La metrica di F-measure supporta tale conclusione; il risultato del modello ComplementNB genera un punteggio più vicino a 1 rispetto a GaussianNB.

In [ ]:
# F-Measure for both classifiers. We expect ComplementNB's score to be higher.
from sklearn.metrics import f1_score

print("Gaussian: {}".format(f1_score(y_test, y_gauss)))
print("Complement: {}".format(f1_score(y_test, y_compl)))